# [SOC-88] Mapping Lab 1: Introduction to Mapping w/ Folium

## Professor David Harding

----------------

<img src="bay_area.png" style="width: 600px;"/>

Welcome! Throughout this lab you will be using Python and packages (such as folium) to create mapping visualizations from the Police Incidents data. The purpose of this lab is to learn how to create elegant data visualizations with maps that convey compelling arguments from data.

**Learning Outcomes:**

By the end of the notebook, students should be able to:

- Use location data to create a map visualization.
- Implement mapping features to convey and create compelling arguments from the data.
- Use aesthetics to communicate information about the data.

## Table of Contents 


1. [Introduction](#1)
<br/><br/>
2. [Introduction to Mapping](#2) 
   - [Tiles](#2.1)
       - [Question 1](#a)
   - [Markers](#2.2)
       - [Question 2](#b)
       - [Question 3](#c)
       - [Question 4](#d)
   - [Circles](#2.3)
       - [Question 5](#e)
       - [Question 6](#f)
<br/><br/>
3. [Conclusion](#3)
4. [Bibliography](#4)

<br><br>
## 1. Introduction <a id='1'></a>

In this lab assignment, you will practice different techniques for creating mapping visualizations using **Folium**. We will be using the same police incidents data from the homework, namely `Police_Incidents_2019.csv`. This data was collected from [Open Data Minneapolis](http://opendata.minneapolismn.gov/).

Run the following cell to load the necessary packages.

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import folium
import json
import os

#Updates the folium package in case you already installed it before
!pip install folium --upgrade -q

<br><br>

## 2. Introduction to Mapping <a id='2'></a>

**Data Mapping** is a process for managing spatial data and creating a type of data visualization in maps (e.g. street, satellite, topological, etc.) to communicate location-related information about the data. Every map conveys a different message and has a unique purpose. Here are some examples and their interpretations:

- **Political Map:** communicates geographical boundaries for states, counties, countries, etc. This also includes roads, cities and major water boundaries such as oceans, rivers and lakes.
- **Heat Map:** using color schemes on maps to communicate high values with bright colors and low values with darker colors. For example, a [weather map](https://www.google.com/search?q=weather+map&hl=en&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjdhcDEhbLoAhWM4J4KHX2aCLoQ_AUoAXoECA0QAw&biw=1440&bih=821#imgrc=OmR67PwUq48WEM) shows high temperature in red and colder temperature in blue.
- **Topographic Map:** provides information on the Earth's surface and elevation using contour lines. Modern topological maps can show this information in 3D like [Google Earth](https://www.google.com/earth/). 
- **Road Map:** communicates roads, streets, and highways like [Google Maps](https://www.google.com/maps)


Other programs such as ArcGIS and QGIS are mapping softwares that allow data analysts to create such data visualizations. However, in this lab we will be using Python and a package known as **Folium** to create road maps. You can find the [Folium Documentation here](https://python-visualization.github.io/folium/) and a [Quickstart Guide here](https://python-visualization.github.io/folium/quickstart.html).

Run the code cell below to load the 2019 Police Incidents Data.

In [ ]:
# Reads the .csv file
incidents = Table().read_table('Police_Incidents_2019.csv')
# Cleans the data in string format
incidents['caseNumber'] = np.array([i.strip(' ') for i in incidents.column(3)])
incidents['offense'] = np.array([i.strip(' ') for i in incidents.column(10)])
incidents['description'] = np.array([i.strip(' ') for i in incidents.column(11)])

incidents.show(5)

The `Police_Incidents_2019.csv` dataset shown above has records of all police incidents that took place in 2019. Its columns contain information such as the latitude-longitude information of incidents, police precinct & neighborhood in which the incident occurred, time & date of the report, type of crime, etc. The latitude and logitude data will help us place the coordinates on a map.

To construct a Map, we first must collect the X and Y coordinates for the center of our map. These coordinates must be taken from our table and placed in the form of a list (i.e. `[Y, X]`). Below, we take the first row of the dataset and place its coordinates into `Folium.Map`.

In [ ]:
# Gets Coordinates for Incident 0 (Row 0)
coordinate = [incidents.column('Y').item(0), incidents.column('X').item(0)]

#Creates Map
point_map = folium.Map(location=coordinate)
point_map

It's not surprising to see an output of a map of Minneapolis since our data is collected form [Open Source Minneapolis](https://python-visualization.github.io/folium/quickstart.html) . However, this map is very zoomed out. If we wanted to zoom in on the exact location of the incident, we can use the parameter `zoom_start` and give it a value indicating how far we want to zoom. The larger the value, the more we zoom in.

Run the code cell below to zoom in on our map for the specific incident.

In [ ]:
# Creates map with zoom 16
point_map = folium.Map(location=coordinate, zoom_start=16)
point_map

<br><br>
## 2.1. Tiles <a id='2.1'></a>

**Tiles** is a feature in Folium that allows you to change the outline/background of the map. The current available built-in tiles are: "OpenStreetMap", "Stamen Terrain", "Stamen Toner", "Stamen Watercolor", "CartoDB positron", and "CartoDB dark_matter".

To do this, we use the parameter `tiles` in our `folium.Map` function. OpenStreetMap is the current default. Run the code cell below to visualize our map, then try replacing the default with the other pre-built-in tiles to visualize different outputs.

In [ ]:
# Creates Open Street Map
point_map = folium.Map(location=coordinate, zoom_start=16, tiles="OpenStreetMap")
point_map

<br><br>
### Question 1 <a id='a'></a>

Now, create a map using the X and Y coordinates of the Case Number `MP201971740`. Create a copy of the `incidents` table with just the one row with that specific case, and assign it to `case` . Use an appropriate zoom and the 'Stamen Terrain' tile. Edit the two code cells below.

In [ ]:
# Table for Case Number MP201971740
case = ...
case.show()

In [ ]:
# Coordinates for Case Number MP201971740
case_coordinate = [..., ...]

# Creates Stamen Terrain Map for the case
case_map = folium.Map(location=case_coordinate, zoom_start=..., tiles=...)
case_map

As you can see above and in the previous examples, it's difficult to see where the incident specifically took place. To resolve this, we can use markers to pinpoint the exact location.

<br><br>
## 2.2. Markers <a id='2.2'></a>

To use markers, we use the `folium.Marker` method with some coordinates. Then, we can add it onto our map using `.addto(map)`. The `Marker` method has the following paramaters:

- **location:** takes in [y, x] coordinates
- **popup:** gives a label to your marker in string format (e.g. "Point A"); appears when clicking on the marker
- **tooltip:** a message to the viewer when hovering the mouse over the map.
- **icon:** gives a specific icon (e.g. cloud, information sign) using folium.Icon(color, icon) (use the documentation to view more information about `folium.Icon`

Let's give our `coordinate` a marker on our map. First, we collect the offense and descripton of the first row from the `incidents` table for the tooltip and popup, respectively. Then, we use the `exclamation-sign` icon along with a `red` color all in `folium.Marker` command. Lastly, we add the marker to our map using `.add_to`. Run the code cell below to visualize the map of this specific case.

In [ ]:
# Gets the offense and description of first row
offense = incidents.column('offense').item(0) 
description = incidents.column('description').item(0) 


# Creates a Marker + Adds to Map
folium.Marker(location=coordinate, popup=description, 
              tooltip=offense, icon=folium.Icon(color='red', icon='exclamation-sign')).add_to(point_map)

point_map

You can also change the icon symbol depending on what message you want to convey. In our case, we used an exclamation sign to indicate a warning of an offense at that location. Folium only supports a limited number of icons. You can find the full available [List of Icons](https://getbootstrap.com/docs/3.3/components/) here.

For colors, there are also a limited number of options:

| | Colors| |
| --- | --- | --- |
| beige | black | blue |
| cadetblue | darkblue | darkgreen |
| darkpurple | darkred | gray |
| green | lightblue | lightgray |
| lightgreen | lightred | orange |
| pink | purple | red |

<br><br>
### Question 2 <a id='b'></a>

Now, using your `case_coordinate` from Question 1, create a marker on the map. Edit the code cell below to give the marker a `description` of the offense as provided in the `incidents` table. Use an appropriate icon and color that best descibes the incident. Lastly, allow the name of the `offense` to appear when hovering the mouse over the marker.

In [ ]:
# Gets the offense and description for Case Number MP201971740
case_offense = ...
case_description = ...

# Creates a Marker + Adds to Map
folium.Marker(location=case_coordinate, popup=...,
              tooltip=..., icon=folium.Icon(color='...', icon='...')).add_to(case_map)

case_map

### Question 3 <a id='c'></a>

To visualize multiple markers on a map, we need to create more markers with their given coordinates. Edit the code cell below to create 3 markers for rows 1, 2, and 3. Give the markers appropriate colors, offense name, descriptions, and icons based on their corresponding incident. The full list of icons is provided [here](https://getbootstrap.com/docs/3.3/components/).

**Note:** When trying out icons from this website into your `icon` parameter, exclude the 'glyphicon' portion of the icon label and copy only the specific name of the icon.

In [ ]:
point_map = folium.Map(location=coordinate, zoom_start=13, tiles="OpenStreetMap")

# For Incident 1 (Row 1)
coordinateA = [..., ...]
offenseA = ...
descriptionA = ...
folium.Marker(location=..., popup=..., 
              tooltip=..., icon=folium.Icon(color='...', icon='...')).add_to(point_map)

# For Incident 2 (Row 2)
coordinateB = [..., ...]
offenseB = ...
descriptionB = ...
folium.Marker(location=..., popup=..., 
              tooltip=..., icon=folium.Icon(color='...', icon='...')).add_to(point_map)

# For Incident 3 (Row 3)
coordinateC = [..., ...]
offenseC = ...
descriptionC = ...
folium.Marker(location=..., popup=..., 
              tooltip=..., icon=folium.Icon(color='...', icon='...')).add_to(point_map)

point_map

### Question 4 <a id='d'></a>

Now, let's create a map with markers for a subset of the coordinates in our `incidents` Table. Assign `downtown_west` to the data points from `incidents` corresponding to the cases where the `neighborhood` is in `Downtown West`. Then, place it onto our map with their corresponding color, offense name, description, and icon. Here, we use a `for` loop to make Python do the repetitive work instead of us manually creating a marker for each Downtown West case. Then, feel free to zoom in and out to see the different incidents. Edit the code cell below.

**Note:** If you do not see any markers, try running the cell again.

In [ ]:
incidents_map = folium.Map(location=coordinate, tiles="OpenStreetMap", zoom_start=15)

# A table for incidents in Downtown West
downtown_west = ...

for i in range(len(downtown_west.rows)):
    coordinate = [..., ...]
    offense = ...
    description = ...
    folium.Marker(location=..., popup=..., 
                  tooltip=..., icon=folium.Icon(color='...', icon='...')).add_to(incidents_map)
    
incidents_map

By using the same `for` loop, we can filter based on other neighborhoods. In addition, we can create other maps for specific incidents (e.g. burglaries, assults, thefts, etc.). We can also map all the incidents that occured within a specific date. All of this can be done by using our `for` loop to make it easier to mark down all the cases and their respective information.

## 2.3. Circles <a id='2.3'></a>

Another way of communicating data within a certain region is using `Circles`. We can create a circular region with a specified radius to show the vicinity of some incidents within the region. To do this, we use two commands:
- `folium.Circle` creates the circle based on the coordinates of the center, and
- `folium.CircleMarker` allows us to design the circle.

Just like with `Markers`, once we have our `Circle`, we then use `.add_to` to add it to our folium map. 

`folium.Circle` and `folium.CircleMarker` have the following parameters:

- **location:** the center coordinates of the circle
- **radius:** the distance from the center
- **popup:** a label to your marker in string format (e.g. "Point A")
- **color:** the color for the outline of the circle
- **fill:** boolean parameter (True or False) to fill the circle with the color specified before

Now, lets create a `Circle` for the 'Bike Theft' incidents in Downtown West. Let's first filter our `incidents` table with only rows whose `neighborhood` is `Downtown West` and `offense` is `BIKETF`.

In [ ]:
# A table of Bike thefts in Downtown West
bike_theft = incidents.where('neighborhood', are.equal_to('Downtown West')).where('offense', are.equal_to('BIKETF'))
bike_theft.show(5)

Let's assume we know the center of this subset of our data. The center is indicated in code cell below.

In [ ]:
# A row for the center of the circle
center = bike_theft.where('caseNumber', are.equal_to('MP2019126982'))
center

### Question 5 <a id='e'></a>

Using the `center`, edit the code cells below to create a circle for Bike Thefts in Downtown West. Test different radii to see which best fits the data (start off with 300 and then edit accordingly). Provide an appropriate color, fill, popup, and tooltip.

In [ ]:
# Get Coordinate, Offense, and Description from Center Coordinate
center_coordinate = [..., ...]
center_offense = ...
center_description = ...

# Do NOT edit the following lines; the following helps set up bike_map for you
bike_map = folium.Map(location=center_coordinate, tiles="OpenStreetMap", zoom_start=15)
for i in range(len(bike_theft.rows)):
    coordinate = [bike_theft['Y'].item(i), bike_theft['X'].item(i)]
    offense = bike_theft['offense'].item(i)
    description = bike_theft['caseNumber'].item(i)
    folium.Marker(location=coordinate, popup=description, 
                  tooltip=offense, icon=folium.Icon(color='red', icon='exclamation-sign')).add_to(bike_map)

# Create your Circle here
folium.Circle(radius=..., 
              location=...,  
              color='...', 
              fill=..., tooltip=...,
             popup=...).add_to(bike_map)
bike_map

### Question 6 <a id='f'></a>

From the map outputted above, what conclusions can someone draw? Who would benefit from this type of information? What information can be conveyed based on your color choice?

**Write your answer here. Double-click to edit the cell.**

<br><br>
<div class="alert alert-warning">
<b> CHALLENGE QUESTION: Use the code cell below to create different maps that convey different information. Some suggestions are listed below:

- A map that shows all the common shoplifting incidents in Downtown West.
- A map showing all the incidents that occured in precinct 3 in April 2019.
- A map with circles showing the vicinity of different incidents (e.g. automobile theft, shopliffting, etc.)
    
Feel free to make your own maps!
    
</b> 
</div>

In [ ]:
# Edit this code cell
# To create your own maps!

<br><br>
## Conclusion <a id='3'></a>

Congratulations! You now have the ropes to create mapping visualizations with the Folium library. You learned how to create a map with coordinates, add markers to indicate specific locations, and draw circles to highlight interesting vicinities.

These are only some tips for creating mapping visualizations. Feel free to explore other aspects that you can create in the Folium Documentation!

<br><br>
## Bibliography <a id='3'></a>

- 2019 Police Incidents Data - Open Data Minneapolis http://opendata.minneapolismn.gov/
- Hobart M. King - Different Types of Maps. https://geology.com/maps/types-of-maps/
- Folium Documentation - https://python-visualization.github.io/folium/
- Folium Quickstart Guide - https://python-visualization.github.io/folium/quickstart.html
- Bootstrap - List of Icons https://getbootstrap.com/docs/3.3/components/

---

Notebook developed by: Elias Saravia

**Data Science Modules:** http://data.berkeley.edu/education/modules

**Data Science Offerings at Berkeley:** https://data.berkeley.edu/academics/undergraduate-programs/data-science-offerings